In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

# load preprocessed data
We will work with a long data frame - categorical columns has one line per combination of `Country Name`, `Country Code`, `years` and `Region` and each row contain 2 values:

* `visitors` - the number of tourist which visited this country in the year
* `receipts` - how much these tourist have spent in the country in that year

We have values for 215 countries for years 1995-2018

Data were preprocessed using - https://github.com/vaclavdekanovsky/data-analysis-in-examples/blob/master/Vizualizations/Plotly/Preprocess/Preprocessing.ipynb

In [2]:
# First run the preprocessing notebook
# https://github.com/vaclavdekanovsky/data-analysis-in-examples/blob/master/Vizualizations/Plotly/Preprocess/Preprocessing.ipynb
# then load the preprocessed pickles
long_df = pd.read_pickle("../Preprocess/long.plk")
yr2018 = long_df[long_df["years"]=="2018"]
spfrit = long_df[long_df["Country Name"].isin(["Spain","France","Italy"])]
sptuar = long_df[long_df["Country Name"].isin(["Spain","Turkey","Aruba"])]

## Simple line chart

In [ ]:
# simple line chart with Plotly Express
px.line(long_df, 
        x="years", 
        y="visitors", 
        color="Country Name", 
        title="Growth of tourism 1995-2018")

We can apply the grey color to all traces using `fig.update_traces({"line":{"color":"lightgrey"}})` and then specify the color of some traces using `.update_traces(patch={})` and `selector={"legendgroup":"Country Name"}`

In [ ]:
fig = px.line(long_df,
              x="years",
              y="visitors",  
              color="Country Name")

# set color of all traces to lightgrey
fig.update_traces({"line":{"color":"lightgrey"}})

# color Turkish line to blue
fig.update_traces(patch={"line":{"color":"blue", "width":5}}, 
                  selector={"legendgroup":"Turkey"})

# color Japanese line to red
fig.update_traces(patch={"line":{"color":"red", "width":5}}, 
                  selector={"legendgroup":"Japan"})

# remove the legend, y-axis and add a title
fig.update_layout(title="Tourism Growth in Turkey and Japan",
                showlegend=False,
                yaxis={"visible":False})
                
fig.show()

If you wonder how I know which values to supply to the dictionary, that `{"line":{"color":"blue", "width":5}` changes the properties of the line and that `legendgroup` is the right parameter to identify the line by `Country name` the easiest way is to read `fig["data"]`. Each Plotly chart is a dictionary and all parameters can be changed when you update this dictionary. 

In [5]:
fig["data"][0]

Scattergl({
    'hovertemplate': 'Country Name=Aruba<br>years=%{x}<br>visitors=%{y}<extra></extra>',
    'legendgroup': 'Aruba',
    'line': {'color': 'lightgrey', 'dash': 'solid'},
    'mode': 'lines',
    'name': 'Aruba',
    'showlegend': True,
    'x': array(['1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
                '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                '2013', '2014', '2015', '2016', '2017', '2018'], dtype=object),
    'xaxis': 'x',
    'y': array([ 619000.,  641000.,  650000.,  647000.,  683000.,  721000.,  691000.,
                 643000.,  642000.,  728000.,  733000.,  694000.,  772000.,  827000.,
                 813000.,  824000.,  869000.,  904000.,  979000., 1072000., 1225000.,
                1102000., 1070500., 1082000.]),
    'yaxis': 'y'
})

## Ordering the lines
But the order of the lines is that Turkey and Japan lie somewhere in the middle, so some grey lines are below it and some are above. 

We need to sort the dataframe (or the traces), so that the highlightted bars are on the top. One option is to use `category_orders` parameter of Plotly. 

In [ ]:
fig = px.line(long_df,
              x="years",
              y="visitors",  
              color="Country Name",
             category_orders={"Country Name":["Japan","Turkey"]})

# set color of all traces to lightgrey
fig.update_traces({"line":{"color":"lightgrey"}})

# color Turkish line to blue
fig.update_traces(patch={"line":{"color":"blue", "width":5}}, 
                  selector={"legendgroup":"Turkey"})

# color Japanese line to red
fig.update_traces(patch={"line":{"color":"red", "width":5}}, 
                  selector={"legendgroup":"Japan"})

# remove the legend, y-axis and add a title
fig.update_layout(title="Tourism Growth in Turkey and Japan with category_order",
                showlegend=False,
                yaxis={"visible":False})
                
fig.show()

But that doesn't lead to the desired output. Japan and Turkey are now on the complete bottom. Other option is to sort the data frame itself by adding a sort column, map values 1,2 to our desired countries and 3 to all the others.

In [7]:
# sort the dataframe
sorted_df = long_df.copy()

# map the value order
sorted_df["order"] = sorted_df["Country Name"].map({"Japan": 1, "Turkey": 2}).fillna(3)

# sort by this order
sorted_df.sort_values(by=["order","years"], ascending=False, inplace=True)
sorted_df.head(3)

,Country Name,Country Code,Region,years,visitors,receipts,order
4945,Aruba,ABW,Latin America & Caribbean,2018,1082000.0,2.024000e+09,3.0
4946,Afghanistan,AFG,South Asia,2018,0.0,5.000000e+07,3.0
4947,Angola,AGO,Sub-Saharan Africa,2018,218000.0,5.570000e+08,3.0


In [ ]:
fig = px.line(sorted_df,
              x="years",
              y="visitors",  
              color="Country Name")

# set color of all traces to lightgrey
fig.update_traces({"line":{"color":"lightgrey"}})

# color Turkish line to blue
fig.update_traces(patch={"line":{"color":"blue", "width":5}}, 
                  selector={"legendgroup":"Turkey"})

# color Japanese line to red
fig.update_traces(patch={"line":{"color":"red", "width":5}}, 
                  selector={"legendgroup":"Japan"})

# remove the legend, y-axis and add a title
fig.update_layout(title="Tourism Growth in Turkey and Japan (sorted dataset)",
                showlegend=False,
                yaxis={"visible":False})
                
fig.show()

Other option how to update the colors of the chart is to provide a dictionary in the format `{trace_name: color}` as an input to `color_discrete_map`. 

In [9]:
sorted_df

# crete a dict with colors:
colors = pd.DataFrame(sorted_df["Country Name"].unique(), columns=["Country Name"])
colors["color"] = colors["Country Name"].map({"Japan": "blue", "Turkey": "red"}).fillna("lightgrey")

# color map is a dict with colors, lightgrey for most, {"Aruba": "lightgrey", ... "Japan: "blue", ...}
color_map = {v["Country Name"]: v["color"] for k,v in colors.iterrows()}

# show sample from the dictionary
{k:color_map[k] for k in color_map if k in ["Aruba","Japan","Turkey","Zimbabwe"]}

{'Aruba': 'lightgrey',
 'Zimbabwe': 'lightgrey',
 'Turkey': 'red',
 'Japan': 'blue'}

In [ ]:

# but still my lines are somewhere in the middle
fig = px.line(sorted_df,
              x="years",
              y="visitors", 
              color="Country Name", 
              line_group="Country Name",
              color_discrete_map=color_map)

fig.update_layout(title="Tourism Growth in Turkey and Japan",
                # remove the legent
                showlegend=False,
                  
                # make y-axis invisible
                yaxis={"visible":False})
                
fig.show()

In [ ]:
for i,d in enumerate(fig["data"]):
    if d["legendgroup"] in ["Japan","Turkey"]:
        fig["data"][i]["line"]["width"] = 5
        
fig.show()

## Anotations
The chart still looks quite ugly. We have removed the legend and the axis so we don't even know how many visitors came to these countries. Let's add some annotations. 

Annotations are added using `fig.update_layoute(annotations=[])` where you supply the list of dictionaries describing the position and the style of the text. You can refer the position to the plot area or to the plot image canvas when you use `xref="paper"`. In such a case (x:0, y:0 is the bottom left corner of the plot area and (x:1, y:1) is the top right one). 

Let's see what you can do with plotly annotations. I think the most interesting is that you refer `x` to canvas and `y` to the coordinate on the chart. 

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(
    title="Annotations' position",
    annotations=\
    [
    {"x":4, "y":0, "text": "<i>(4,0)</i>", "showarrow":False},
    {"x":4, "y":2, "text": "(4,2)", "showarrow":True, 
     "font":{"color":"blue", "size":15}, "arrowcolor":"blue"},
    {"x":0, "y":2, "text": "<b>(0,2) with ax and arrowhead</b>", 
     "showarrow":True, "arrowhead": 3, "ax":40},
    {"xref": "paper", "x":0, "y":0, "text": "<b>paper (0,0)</b>", "showarrow":False},
    {"xref": "paper", "xanchor": "right","x":0, "y":-.5, 
     "text": "<b>(paper: 0,plot: -.5), mix paper <br>and plot positioning</b>", "showarrow":False},
    {"xref": "paper", "yref": "paper",  "xanchor": "left", "yanchor":"bottom","x":1, "y":1, 
     "text": "<b>paper (1,1), leftbottom anchor</b>", "showarrow":False},
    {"xref": "paper", "yref": "paper",  "xanchor": "right", "yanchor":"top","x":1, "y":1, 
     "text": "<b>paper (1,1), righttop anchor</b>", "showarrow":False},
    {"xref": "paper",  "xanchor": "center", "yanchor":"middle","x":1, "y":0, 
     "text": "(paper: 1, plot: 0) <br> centermiddle anchor", "showarrow":False},
    ],
    margin={"l":300, "r": 250},)
fig.show()

Let's create the list of dictionaries with the annotations at the beginning and the end of the lines.

In [13]:
# annotation at the beginning of the line 
# use the "xanchor": "right" so that the labels stick to the right side of the plot area
turkey_annotation = \
[{"xref":"paper", "yref":"paper", "x":0, "y":0.15,
  "xanchor":'right', "yanchor":"top",
  "text":'7M',
  "font":dict(family='Arial', size=15, color="red"),
  "showarrow":False},
  # end of the line legend
  # use the "xanchor": "left" so that the labels stick to the right side of the plot area
  {"xref":"paper", "yref":"paper", "x":1, "y":0.53,
   "xanchor":"left", "yanchor":"top",
   "text":'Turkey (45M)',
   "font":dict(family='Arial', size=15, color="red"),
   "showarrow":False},]

japan_annotation = [{"xref":"paper", "yref":"paper", "x":0, "y":0.1,
                    "xanchor":'right', "yanchor":'top',
                    "text":'3.3M',
                    "font":dict(family='Arial',
                                size=15,
                                color="blue"),
                    "showarrow":False}, 
                   {"xref":"paper", "yref":"paper", "x":1, "y":0.39,
                    "xanchor":'left', "yanchor":'top',
                    "text":'Japan (31M)',
                    "font":dict(family='Arial',
                                size=15,
                                color="blue"),
                    "showarrow":False}]



In [ ]:
fig = px.line(sorted_df,
              x="years",
              y="visitors", 
              color="Country Name", 
              line_group="Country Name",
              color_discrete_map=color_map)

for legendgroup in ["Turkey","Japan"]:
    fig.update_traces(patch={"line":{"width":5}}, selector={"legendgroup":legendgroup})

fig.update_layout(title="Tourism Growth in Turkey and Japan",
                # remove the legent
                showlegend=False,
                  
                # make y-axis invisible
                yaxis={"visible":False},
                
                # create the annoations
                # point annotattion
                annotations=turkey_annotation+japan_annotation,
                 
                # adjust the margin, so that annotations fit into the canvas
                margin={"l":50, "r": 100})
fig.show()

Cool. We can also refer the `y` using `(x: 0 on canvas and y: 7_000_000 on the plot)`. Also using `xnachor` and `yanchor` let us position the text quite quickly and precisely. Don't forget to style the texts with color and size. 

In [15]:
# annotation at the beginning of the line 
# use the "xanchor": "right" so that the labels stick to the right side of the plot area
annotation_turkey = \
[{"xref": "paper", "x":0, "y":7_000_000,
  "xanchor":'right', "yanchor":"middle",
  "text":'7M',
  "font":dict(family='Arial', size=15, color="red"),
  "showarrow":False},
  # end of the line legend
  # use the "xanchor": "left" so that the labels stick to the right side of the plot area
  {"xref":"paper", "x":1, "y":45_000_000,
   "xanchor":"left", "yanchor":"middle",
   "text":'Turkey (45M)',
   "font":dict(family='Arial', size=15, color="red"),
   "showarrow":False},]

annotation_japan = [{"xref": "paper", "x":0, "y":3_300_000,
                    "xanchor":'right', "yanchor":'middle',
                    "text":'3.3M',
                    "font":dict(family='Arial',
                                size=15,
                                color="blue"),
                    "showarrow":False}, 
                   {"xref":"paper", "x":1, "y":31_000_000,
                    "xanchor":'left', "yanchor":'middle',
                    "text":'Japan (31M)',
                    "font":dict(family='Arial',
                                size=15,
                                color="blue"),
                    "showarrow":False}]



Annotations can be used for at least 4 puposes:
    
* To highlight point(s)
* To describe/highlight an area
* To label a desired point (usually outside of the chart)
* Instead of a legend

We used a label on the begining of the lines and the legend at the end of the lines. Let's add the point highlight and area description. 

In [16]:
tourism_boom_in_japan_annotation = [{"x":2011, "y":6220000, "ay": -40, 
                        "text": "<b>Tourism Boom<br> in Japan 2011</b>",
                        "arrowhead": 3, "showarrow":True,
                        "font": {"size": 15}, "bgcolor":None, "textangle": 0}]

area_annotation =  [{"x":2007, "y":40000000, 
                        "text": "<b>Number of tourist is growing</b>",
                         "textangle": -25,
                        "showarrow":False,
                         "bgcolor":"lightblue",
                        "font": {"size": 15}}]

In [ ]:
fig = px.line(sorted_df,
              x="years",
              y="visitors", 
              color="Country Name", 
              line_group="Country Name",
              color_discrete_map=color_map)

for legendgroup in ["Turkey","Japan"]:
    fig.update_traces(patch={"line":{"width":5}}, selector={"legendgroup":legendgroup})

fig.update_layout(title="Tourism Growth in Turkey and Japan",
                # remove the legent
                showlegend=False,
                  
                # make y-axis invisible
                yaxis={"visible":False},
                
                # create the annoations
                # point annotattion
                annotations=annotation_turkey+annotation_japan+tourism_boom_in_japan_annotation+area_annotation,
                 
                # adjust the margin, so that annotations fit into the canvas
                margin={"l":50, "r": 100})
fig.show()

## Buttons
Plotly contains many interactive features and button which let you control the drawn elements are one of the them. 

In [ ]:
 
# but still my lines are somewhere in the middle
fig = px.line(sorted_df,
              x="years",
              y="visitors", 
              color="Country Name", 
              line_group="Country Name")

# for the interactivity we need to know which traces belong to each country
traces = {}
for i,d in enumerate(fig.to_dict()["data"]):
    traces[d["legendgroup"]] = i
        

fig.update_traces({"line":{"color":"lightgrey"}})

fig.update_layout(title="Tourism Growth in Turkey and Japan",
                # remove the legent
                showlegend=False,
                  
                # make y-axis invisible
                yaxis={"visible":False},               
                
                
                # create big enough margins for our annotations
                margin={"l":50, "r": 100},
                 

                # button
    updatemenus=[
        # a dropdown `direction="down"`
        # changing the color of the bars
        dict(
            buttons=list([
                dict(
                   args=[
                       {"line.color":["blue"],"line.width":5}, 
                         {"title":"Tourism growth in Japan",
                           "annotations": annotation_japan}, [traces["Japan"]]],    
                    # args2 used as toggle button, applies when clicked on active button
                    args2=[{"line.color":["lightgrey"],"line.width":2},
                           {"title":"Tourism growth in Japan and Turkey",
                            "annotations": None}],
                    label="Japan",
                    method="update",)
                ,dict(
                    args=[{"line.color":["red"],"line.width":5}, 
                         {"title":"Tourism growth in Turkey",
                          "annotations": annotation_turkey}, [traces["Turkey"]]],  
                    args2=[{"line.color":["lightgrey"],"line.width":2},
                           {"title":"Tourism growth in Japan and Turkey",
                            "annotations": None}],
                    label="Turkey",
                    method="update")
               
            ]),
            type = "buttons",
            direction="left",
            pad={"r": 10, "t": 10},
            showactive=False,
            x=0.5,
            xanchor="left",
            y=1.25,
            yanchor="top"
        )]
    )


        

fig.show()

If you click on both Japan and Turkey buttons, without triggering the toggle-clean action, you will see both red and blue line. I did not achive it anyhow, even when I supplied a list of 215 style dictionaries for each country. You have to use Plotly's dashboard library with callback actions which allow almost every imaginable output.

In [19]:
reset = {"line.color":["lightgrey"],"line.width":2}
japan = {"line.color":["blue"],"line.width":5}
turkey = {"line.color":["red"],"line.width":5}
jp_list = []
tr_list = []
for i,d in enumerate(fig.to_dict()["data"]):
    if d["legendgroup"] == "Japan":
        jp_list.append(japan)
    else: 
        jp_list.append(reset)
        
    if d["legendgroup"] == "Turkey":
        tr_list.append(turkey)
    else:

        tr_list.append(reset)

# IIHF standings
This type of chart is useful for displaying changes in the ranks. Let's explore the best Ice-Hockey national teams over last 6 years.
I have switched to Linux and manually copied these rankings from the wiki - https://en.wikipedia.org/wiki/IIHF_World_Ranking into ods spredsheet format. In order to read it in pandas, I have to install `pip install odfpy`

In [20]:
iihf = pd.read_excel("IIHF_Standings.ods", engine="odf")
sweden = iihf[iihf["Country"]=="Sweden"]
iihf.sample(3)

,Country,Year,Ranking
31,Czech Rep.,2019,5
20,Finland,2018,5
1,Canada,2019,1


In [ ]:
fig = px.line(iihf, x="Year", y="Ranking", color="Country", 
              category_orders={"Country":["Sweden"]})
fig.update_traces({"line":{"color":"lightgrey"}})
fig.update_traces(patch={"line":{"color":"blue", "width":5}}, 
                  selector={"legendgroup":"Sweden"})
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
fig = px.line(iihf, x="Year", y="Ranking", color="Country",
              category_orders={"Country":["Canada","Russia","Finland","Czech Rep.", "USA"]},
             title="IIHF Ranking of Sweden (using update_traces)")
# set all traces to grey
fig.update_traces({"line":{"color":"lightgrey"}})

# set one trace to blue
fig.update_traces(patch={"line":{"color":"yellow", "width":5}, 
                         "text":sweden["Ranking"].values,
                        "textposition":"top center",
                        "mode":"lines+text",
                         "textfont":{"size":15, "color":"blue"},
                         
                        }, 
                  selector={"legendgroup":"Sweden"})

# reverse the axis so that 1 is on the top
fig.update_yaxes(autorange="reversed")

fig.show()

or you can do the same when you prepare the dictionary with the annotations

In [23]:
annotations = []
for index, row in sweden.reset_index().iterrows():
    d = {"x": row["Year"], 
                        "y": row["Ranking"], 
                        "yshift": 6,
                        "text": row["Ranking"],
                        "xanchor": "center", 
                        "yanchor": "bottom", 
                        "showarrow":False,
         "bgcolor": "rgba(0,0,255,.6)",
                       "font":{"color":"yellow", "size": 15}}
    if index == 0: d["xshift"] = -10
    if index == len(sweden)-1: d["xshift"] = 10
    
    annotations.append(d)
    

In [ ]:
fig = px.line(iihf, x="Year", y="Ranking", color="Country",
              category_orders={"Country":["Canada","Russia","Finland","Czech Rep.", "USA"]},
             title="IIHF Ranking of Sweden (using annotations)")

# reverse the axis so that 1 is on the top
fig.update_yaxes(autorange="reversed")

# set all traces to grey
fig.update_traces({"line":{"color":"lightgrey"}})

# set one trace to blue
fig.update_traces(patch={"line":{"color":"blue", "width":5}}, 
                  selector={"legendgroup":"Sweden"})

fig.update_layout(annotations=annotations)



fig.show()